In [3]:
gabe_dates = [
  'January 15', 'January 4',
  'July 13',    'July 24',   'July 30',
  'March 13',   'March 24',
  'May 11',     'May 17',    'May 30']

cheryl_dates = DATES = {
     'May 15',    'May 16',    'May 19',
    'June 17',   'June 18',
    'July 14',   'July 16',
  'August 14', 'August 15', 'August 17'}

In [2]:
# Albert and Bernard just became friends with Cheryl, and they want to know when her birthday is. 
# Cheryl gives them a set of 10 possible dates:



def month(date): return date.split()[0]

def day(date):   return date.split()[1]

# Cheryl then tells Albert and Bernard separately 
# the month and the day of the birthday respectively.

def tell(part):
    "Cheryl tells a part of her birthdate; return a subset of DATES that match the part."
    return {date for date in DATES if part in date}

def know(possible_dates):
    "A person knows the birthdate if they know there is exactly one possible date."
    return len(possible_dates) == 1

def hear(possible_dates, *statements):
    "Return the subset of possible dates that are consistent with all the statements."
    return {date for date in possible_dates
            if all(stmt(date) for stmt in statements)}

# Albert and Bernard make three statements:

def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(month(date))
    return (not know(after_being_told) 
            and all(not know(tell(day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    at_first = tell(day(date))
    return (not know(at_first)
            and know(hear(at_first, albert1)))

def albert2(date):
    "Albert: Then I also know when Cheryl's birthday is."
    return know(hear(tell(month(date)), bernard1))
    
# So when is Cheryl's birthday?

def cheryls_birthday(dates):
    "Return a list of the possible dates after hearing the three statements."
    return hear(using(dates), albert1, bernard1, albert2)

def using(dates):
    "Make dates be the value of the global variable DATES."
    global DATES # This is necessary because `tell` looks at `DATES`
    DATES = dates
    return dates

# Some tests

assert month('May 19') == 'May'
assert day('May 19') == '19'
assert albert1('May 19') == False
assert albert1('July 14') == True
assert know(tell('17')) == False
assert know(tell('19')) == True

In [5]:
print(cheryls_birthday(gabe_dates))
print(cheryls_birthday(cheryl_dates))

{'July 30'}
{'July 16'}


In [9]:
some_dates = {mo + ' ' + d1 + d2
              for mo in ('March', 'April', 'May', 'June', 'July')
              for d1 in '12'
              for d2 in '3456789'}

import random

def pick_dates(puzzle=cheryls_birthday, k=10):
    "Pick a set of dates for which the puzzle has a unique solution."
    while True:
        dates = set(random.sample(some_dates, k))
        solutions = puzzle(dates)
        if know(solutions):
            return solutions.pop(), dates

In [10]:
pick_dates()

('March 18',
 {'July 24',
  'June 19',
  'June 27',
  'March 18',
  'March 19',
  'March 27',
  'May 18',
  'May 25',
  'May 26',
  'May 29'})

In [35]:
def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(month(date))
    return (not know(after_being_told) 
            and all(not know(tell(day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    return not know(hear(tell(day(date)), albert1))

def albert2(date):
    "Albert: I still don't know."
    return not know(hear(tell(month(date)), bernard1))

def bernard2(date):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    return not know(hear(tell(day(date)), albert2))

def albert3(date):
    "Albert: OK, now I know."
    return know(hear(tell(month(date)), bernard2))

def bernard3(date):
    "Bernard: OK, now I know."
    return know(hear(tell(day(date)), albert3))

In [36]:
def cheryls_birthday_complex(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return hear(using(dates), albert1, bernard1, albert2, bernard2, albert3, bernard3)

In [39]:
output = pick_dates(puzzle=cheryls_birthday_complex)
result, dates = output[0], output[1]
print(result)
print(dates)


May 18
{'June 23', 'April 19', 'May 26', 'April 23', 'July 25', 'June 18', 'March 26', 'May 18', 'June 19', 'March 25'}


In [40]:
using(dates)
for d in dates:
    if albert1(d):
        print()


June 23
April 19
May 26
April 23
June 18
March 26
May 18
June 19
March 25
